In [1]:
%load_ext Cython

In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as pyplot
import optuna as op
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Utils.createURM import createURM
from Utils.createICM import createICM
from Utils.combineURMICM import combine

### URM

In [3]:
URM = createURM()
ICM = createICM()

/Users/matteopancini/PycharmProjects/recsys-challenge-2022-Pancini-Vitali/Utils/createURM.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../Input/interactions_and_impressions.csv')
/Users/matteopancini/PycharmProjects/recsys-challenge-2022-Pancini-Vitali/Utils/createICM.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  URM = pd.read_csv('../Input/interactions_and_impressions.csv')


In [4]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

In [5]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2711 ( 6.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1520 ( 3.7%) Users that have less than 1 test interactions


In [6]:
URMICMCombined = combine(ICM=ICM, URM = URM)

### SLIM

In [7]:
recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URMICMCombined.T, verbose=False)

def objective(trial):
    alpha = trial.suggest_float("alpha", 0, 1)
    l1_ratio = trial.suggest_float("l1_ratio", 0, 1)
    topK = trial.suggest_float("topK", 100, 500)
    recommender.fit(alpha=alpha, l1_ratio=l1_ratio, topK=int(topK))
    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    return result_dict.loc[10]["MAP"]

In [8]:
study = op.create_study(direction='minimize')
study.optimize(objective, n_trials=15)

[I 2022-11-04 08:44:39,727] A new study created in memory with name: no-name-d9839c4a-5f03-459b-9690-90a5199154f0
100%|█████████▉| 24504/24507 [01:11<00:00, 345.04it/s]


EvaluatorHoldout: Processed 38918 (100.0%) in 12.82 sec. Users per second: 3035


[I 2022-11-04 08:46:03,661] Trial 0 finished with value: 0.0 and parameters: {'alpha': 0.7144444121186274, 'l1_ratio': 0.8941505797892362, 'topK': 193.62943556987977}. Best is trial 0 with value: 0.0.
100%|█████████▉| 24504/24507 [02:05<00:00, 195.83it/s]


EvaluatorHoldout: Processed 38918 (100.0%) in 13.03 sec. Users per second: 2986


[I 2022-11-04 08:48:21,889] Trial 1 finished with value: 0.0 and parameters: {'alpha': 0.9355381247279914, 'l1_ratio': 0.7262796263865665, 'topK': 200.43803329859162}. Best is trial 0 with value: 0.0.
100%|█████████▉| 24504/24507 [16:06<00:00, 25.36it/s]


...

EvaluatorHoldout: Processed 38918 (100.0%) in 13.09 sec. Users per second: 2973


[I 2022-11-04 09:14:28,882] Trial 4 finished with value: 0.0 and parameters: {'alpha': 0.0915506719173772, 'l1_ratio': 0.8998093656529678, 'topK': 441.94499702724914}. Best is trial 0 with value: 0.0.
100%|█████████▉| 24504/24507 [21:12<00:00, 19.26it/s]  


EvaluatorHoldout: Processed 38918 (100.0%) in 12.96 sec. Users per second: 3004


[I 2022-11-04 09:35:54,362] Trial 5 finished with value: 0.0 and parameters: {'alpha': 0.917956893563923, 'l1_ratio': 0.05424945011007676, 'topK': 499.39665667749887}. Best is trial 0 with value: 0.0.
  1%|          | 168/24507 [00:03<04:10, 97.08it/s][W 2022-11-04 09:35:57,535] Trial 6 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/Users/matteopancini/opt/anaconda3/envs/recsys/lib/python3.9/multiprocessing/pool.py", line 853, in next
    item = self._items.popleft()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/matteopancini/opt/anaconda3/envs/recsys/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/tt/9v6608x550n51759gpcrffsc0000gn/T/ipykernel_61283/1017296203.py", line 7, in objective
    recommender.fit(alpha=alpha, l1_ratio=l1_ratio, 

KeyboardInterrupt: 

In [9]:
alpha = study.best_params['alpha']
l1_ratio = study.best_params['l1_ratio']
topK = study.best_params['topK']

In [10]:
alpha

0.7144444121186274

In [11]:
l1_ratio

0.8941505797892362

In [12]:
topK

193.62943556987977

In [14]:
recommender.fit(alpha=0.8804905330217533, l1_ratio=0.9120820630476552, topK=int(491.2852695850108))
result_df, _ = evaluator_test.evaluateRecommender(recommender)
result_df

100%|█████████▉| 24504/24507 [00:42<00:00, 570.40it/s]


EvaluatorHoldout: Processed 40035 (100.0%) in 13.33 sec. Users per second: 3002


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.000087,0.000205,0.000198,0.000024,0.000074,0.000242,0.000151,0.000121,0.000874,0.000242,...,0.961709,0.000841,0.961709,0.000503,3.791113,0.91992,0.00115,0.287535,0.044782,0.738861


In [13]:
recommender.fit(alpha=0.7144444121186274, l1_ratio=0.8941505797892362, topK=int(193.62943556987977))
result_df, _ = evaluator_test.evaluateRecommender(recommender)
result_df


100%|█████████▉| 24504/24507 [01:11<00:00, 342.34it/s]


EvaluatorHoldout: Processed 40109 (100.0%) in 13.47 sec. Users per second: 2977


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.963487,0.0,0.963487,0.000591,4.095323,0.928331,0.001274,0.308715,0.047869,0.450896
